In [20]:

chr(0)

'\x00'

In [21]:
print(chr(0))

 


In [22]:
"this is a test" + chr(0) + "string"

'this is a test\x00string'

In [23]:
print("this is a test" + chr(0) + "string")

this is a test string


In [24]:
test_string = "hello! こんにちは!"
utf8_encoded = test_string.encode("utf-8")
print(utf8_encoded)
print(list(utf8_encoded))
print(utf8_encoded.decode("utf-8"))

b'hello! \xe3\x81\x93\xe3\x82\x93\xe3\x81\xab\xe3\x81\xa1\xe3\x81\xaf!'
[104, 101, 108, 108, 111, 33, 32, 227, 129, 147, 227, 130, 147, 227, 129, 171, 227, 129, 161, 227, 129, 175, 33]
hello! こんにちは!


In [25]:
a = b'\xe3\x41'
a.decode("utf-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe3 in position 0: invalid continuation byte

In [132]:
b'\xe2'.decode("utf-8")

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xe2 in position 0: unexpected end of data

In [ ]:
print(len(list(test_string.encode("utf-8"))))
print(len(list(test_string.encode("utf-16"))))


23
28


In [ ]:
def decode_utf8_bytes_to_str_wrong(bytestring: bytes):
    return "".join([bytes([b]).decode("utf-8") for b in bytestring])
decode_utf8_bytes_to_str_wrong("hello".encode("utf-8"))

'@hello'

In [ ]:
import regex as re

PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
re.findall(PAT, "some text that i'll pre-tokenize")
for i in re.finditer(PAT, "some text that i'll pre-tokenize"):
    print(i)

<regex.Match object; span=(0, 4), match='some'>
<regex.Match object; span=(4, 9), match=' text'>
<regex.Match object; span=(9, 14), match=' that'>
<regex.Match object; span=(14, 16), match=' i'>
<regex.Match object; span=(16, 19), match="'ll">
<regex.Match object; span=(19, 23), match=' pre'>
<regex.Match object; span=(23, 24), match='-'>
<regex.Match object; span=(24, 32), match='tokenize'>


In [26]:
import regex as re

text = '''low low low low low
lower lower widest widest widest
newest newest newest newest newest newest'''
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
word_lst = re.findall(PAT, text)
word_lst

['low',
 ' low',
 ' low',
 ' low',
 ' low',
 '\n',
 'lower',
 ' lower',
 ' widest',
 ' widest',
 ' widest',
 '\n',
 'newest',
 ' newest',
 ' newest',
 ' newest',
 ' newest',
 ' newest']

In [3]:
import regex as re
import collections
#from cs336_basics.pretokenization_example import find_chunk_boundaries
from tqdm import trange
import multiprocessing
from typing import IO, Any, BinaryIO, List, Tuple
from worker import process_chunk

In [ ]:
vocab_size = 320000
num_processes = 16
input_path = 'data/owt_train.txt'
## Usage

with open(input_path, "rb") as f:
    boundaries = find_chunk_boundaries(
        f, num_processes, "<|endoftext|>".encode("utf-8"))
special_tokens = ["<|endoftext|>"]
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
chunk_definitions = list(zip(boundaries[:-1], boundaries[1:]))
job_args = [(input_path, start, end, special_tokens, PAT) for start, end in chunk_definitions]
total_counts = collections.Counter()
with multiprocessing.Pool(processes=num_processes) as pool:
    # pool.starmap 会将 job_args 中的每个元组解包作为 process_chunk 的参数
    # 它会阻塞，直到所有进程完成任务
    results = pool.starmap(process_chunk, job_args)
    
    # 聚合所有子进程返回的结果
    print("所有进程已完成，正在聚合结果...")
    for chunk_counts in results:
        total_counts.update(chunk_counts)
    

initial__bytes = {i: bytes([i]) for i in range(256)}

#split_pattern = "|".join(re.escape(token) for token in special_tokens)
#text_split = re.split(split_pattern, chunk)
#PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
#for text_item in text_split:
#    word_counts = collections.Counter(m.group() for m in re.finditer(PAT, text_item))
#   total_counts.update(word_counts)
#print(total_counts)

pair_counts = collections.defaultdict(int)
vocab = collections.defaultdict()
for key, value in total_counts.items():
    alpha_list = [bytes([b]) for b in key]
    if len(alpha_list) < 2:
        continue
    vocab[key] = (alpha_list, value)
    for index1, index2 in zip(alpha_list, alpha_list[1:]):
        pair_counts[(index1, index2)] += value
print(pair_counts)

vocab_cur = len(special_tokens) + len(initial__bytes) #未合并前是特殊token数+初始字符数（256）
merges = []
#while vocab_cur < vocab_size: #持续合并
for _ in trange(vocab_size - vocab_cur):
    #pair = max(pair_counts, key=pair_counts.get) #找到频率最高的字符对（还没实现相同频率下优先选哪个）
    pair = max(pair_counts, key=lambda p: (pair_counts[p], p))
    del pair_counts[pair] #删除频率最高的
    merge_alp = pair[0] + pair[1]
    merges.append((pair[0], pair[1])) #记录合并

    for key in list(vocab.keys()):
        alp_list, alp_count = vocab[key]
        i = 0
        while i < len(alp_list) - 1:
            cur_pair = (alp_list[i], alp_list[i+1])
            if cur_pair == pair: #因为合并需要减少左右字符配对频率
                if i > 0:
                    l_alp = alp_list[i-1]
                    l_pair = (l_alp, alp_list[i])
                    pair_counts[l_pair] -= alp_count
                    pair_counts[(l_alp,merge_alp)] += alp_count
                if i + 2 < len(alp_list):
                    r_alp = alp_list[i+2]
                    r_pair = (alp_list[i+1], r_alp)
                    pair_counts[r_pair] -= alp_count
                    pair_counts[(merge_alp, r_alp)] += alp_count
                
                alp_list[i:i+2] = [merge_alp]
            else:
                i += 1
        #更新vocab中key的alp_list
        vocab[key]= (alp_list, alp_count)
    #vocab_cur += 1

final_tokens = initial__bytes.copy()
nxt_id = len(final_tokens)
#添加特殊字符
for s_token in special_tokens:
    final_tokens[nxt_id] = s_token.encode("utf-8")
    nxt_id += 1
#添加训练得到的字符组合
for mer_pair in merges:
    merged = mer_pair[0] + mer_pair[1] 
    final_tokens[nxt_id] = merged
    nxt_id += 1
print(final_tokens)


In [137]:
merges

[(b'h', b'e'),
 (b' ', b't'),
 (b' ', b'a'),
 (b' ', b's'),
 (b' ', b'w'),
 (b'n', b'd'),
 (b' t', b'he'),
 (b'e', b'd'),
 (b' ', b'b'),
 (b' t', b'o'),
 (b' a', b'nd'),
 (b' ', b'h'),
 (b' ', b'f'),
 (b'i', b'n'),
 (b' w', b'a'),
 (b' ', b'T'),
 (b'i', b't'),
 (b'r', b'e'),
 (b'o', b'u'),
 (b' ', b'l'),
 (b' ', b'd'),
 (b' ', b'c'),
 (b' ', b'p'),
 (b'a', b'y'),
 (b' wa', b's'),
 (b'e', b'r'),
 (b' ', b'm'),
 (b'o', b'm'),
 (b' ', b'he'),
 (b' T', b'he'),
 (b'i', b's'),
 (b' ', b'n'),
 (b'o', b'n'),
 (b'a', b'r'),
 (b'i', b'm'),
 (b' s', b'a'),
 (b'l', b'l'),
 (b'i', b'd'),
 (b' h', b'a'),
 (b' ', b'g'),
 (b' ', b'S'),
 (b'a', b't'),
 (b'in', b'g'),
 (b'o', b't'),
 (b'e', b'n'),
 (b'a', b'n'),
 (b'l', b'e'),
 (b'o', b'r'),
 (b'i', b'r'),
 (b' ', b'H'),
 (b'a', b'm'),
 (b'e', b't'),
 (b' ', b'it'),
 (b' t', b'h'),
 (b'i', b'g'),
 (b' The', b'y'),
 (b'i', b'l'),
 (b' ', b'in'),
 (b' H', b'e'),
 (b' p', b'l'),
 (b' ', b'"'),
 (b'o', b'w'),
 (b'v', b'er'),
 (b'r', b'i'),
 (b' ', b'u'),
 (

In [138]:
final_tokens

{0: b'\x00',
 1: b'\x01',
 2: b'\x02',
 3: b'\x03',
 4: b'\x04',
 5: b'\x05',
 6: b'\x06',
 7: b'\x07',
 8: b'\x08',
 9: b'\t',
 10: b'\n',
 11: b'\x0b',
 12: b'\x0c',
 13: b'\r',
 14: b'\x0e',
 15: b'\x0f',
 16: b'\x10',
 17: b'\x11',
 18: b'\x12',
 19: b'\x13',
 20: b'\x14',
 21: b'\x15',
 22: b'\x16',
 23: b'\x17',
 24: b'\x18',
 25: b'\x19',
 26: b'\x1a',
 27: b'\x1b',
 28: b'\x1c',
 29: b'\x1d',
 30: b'\x1e',
 31: b'\x1f',
 32: b' ',
 33: b'!',
 34: b'"',
 35: b'#',
 36: b'$',
 37: b'%',
 38: b'&',
 39: b"'",
 40: b'(',
 41: b')',
 42: b'*',
 43: b'+',
 44: b',',
 45: b'-',
 46: b'.',
 47: b'/',
 48: b'0',
 49: b'1',
 50: b'2',
 51: b'3',
 52: b'4',
 53: b'5',
 54: b'6',
 55: b'7',
 56: b'8',
 57: b'9',
 58: b':',
 59: b';',
 60: b'<',
 61: b'=',
 62: b'>',
 63: b'?',
 64: b'@',
 65: b'A',
 66: b'B',
 67: b'C',
 68: b'D',
 69: b'E',
 70: b'F',
 71: b'G',
 72: b'H',
 73: b'I',
 74: b'J',
 75: b'K',
 76: b'L',
 77: b'M',
 78: b'N',
 79: b'O',
 80: b'P',
 81: b'Q',
 82: b'R',
 83: b'

In [127]:
a = {
    ('a','b'):11,
    ('b','c'):10,
}


b = max(a, key=lambda p: (a[p], p))
b

('a', 'b')

In [133]:
b'\xfb'.decode()

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xfb in position 0: invalid start byte

In [24]:
PAT = r"""'(?:[sdmt]|ll|ve|re)| ?\p{L}+| ?\p{N}+| ?[^\s\p{L}\p{N}]+|\s+(?!\S)|\s+"""
text = 'hello 你好 233'

c = [[bytes([b]) for b in word.encode("utf-8")] for word in re.findall(PAT, text)]
c

[[b'h', b'e', b'l', b'l', b'o'],
 [b' ', b'\xe4', b'\xbd', b'\xa0', b'\xe5', b'\xa5', b'\xbd'],
 [b' ', b'2', b'3', b'3']]

In [23]:
word = "你"

# 你的写法：
[i.encode("utf-8") for i in word]
# → ["你".encode("utf-8")] = [b'\xe4\xbd\xa0'] ← 这是一个多字节 token，不符合 byte-level

# 推荐写法：
[bytes([b]) for b in word.encode("utf-8")]

[b'\xe4', b'\xbd', b'\xa0']

In [27]:
a = [2,4,6,5, 7,10]
is_even = all((n := i) % 2 == 0 for i in a)
n

5

In [28]:
a = ['a','  d']
''.join(a)

'a  d'

In [ ]:
from tests.common import FIXTURES_PATH, gpt2_bytes_to_unicode
from tests.test_tokenizer import get_tokenizer_from_vocab_merges_path


VOCAB_PATH = FIXTURES_PATH / "gpt2_vocab.json"
MERGES_PATH = FIXTURES_PATH / "gpt2_merges.txt"



text_tokenizer = get_tokenizer_from_vocab_merges_path(
        vocab_path=VOCAB_PATH, merges_path=MERGES_PATH, special_tokens=["<|endoftext|>"]
    )
text_tokenizer.encode("<|endoftext|>")

[50256]

In [2]:
special_tokens = ["<|endoftext|>"]
sorted(special_tokens, key=len, reverse=True)

['<|endoftext|>']

In [23]:
from utils.chunking import find_chunk_boundaries
import regex as re

special_tokens = ["<|endoftext|>"]
corpus_path = FIXTURES_PATH / "tinystories_sample.txt"
with open(corpus_path) as f:
    corpus_contents = f.read()
boundaries = find_chunk_boundaries(corpus_contents, 16, special_tokens[0]) #按照第一个special_token分块
chunk_definitions = list(zip(boundaries[:-1], boundaries[1:]))
chunk_definitions

[(0, 728), (728, 1404), (1404, 1932), (1932, 2803), (2803, 3772), (3772, 3786)]

In [24]:
chunk_text = corpus_contents[728:1404]
split_pattern = "|".join(re.escape(token) for token in special_tokens)
text_split = re.split(f"({split_pattern})", chunk_text)
text_split

['',
 '<|endoftext|>',
 '\nOnce upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They all loved to play and swim together.\nOne day, Ollie\'s mom said, "Ollie, hurry and get some fish for dinner!" Ollie swam fast to catch fish. He saw his friend, the duck. "Hi, Ollie!" said the duck. "Hi, duck!" said Ollie. "I need to hurry and catch fish for my family."\nWhile Ollie was catching fish, he found a big shiny stone. He thought, "This is not a fish, but it is so pretty!" Ollie took the shiny stone home to show his family. They all looked at the shiny stone and smiled. The shiny stone made everyone happy, and they forgot about the fish for dinner.\n']

In [25]:
chunk_text

'<|endoftext|>\nOnce upon a time, there was a reliable otter named Ollie. He lived in a river with his family. They all loved to play and swim together.\nOne day, Ollie\'s mom said, "Ollie, hurry and get some fish for dinner!" Ollie swam fast to catch fish. He saw his friend, the duck. "Hi, Ollie!" said the duck. "Hi, duck!" said Ollie. "I need to hurry and catch fish for my family."\nWhile Ollie was catching fish, he found a big shiny stone. He thought, "This is not a fish, but it is so pretty!" Ollie took the shiny stone home to show his family. They all looked at the shiny stone and smiled. The shiny stone made everyone happy, and they forgot about the fish for dinner.\n'

In [26]:
split_pattern

'<\\|endoftext\\|>'

In [8]:
import torch
from einops import rearrange, einsum

device = torch.device("mps" if torch.backends.mps.is_available() else "cpu")
images = torch.randn(64, 128, 128, 3, device=device) # (batch, height, width, channel)
dim_by = torch.linspace(start=0.0, end=1.0, steps=10, device=device)
## Reshape and multiply

## Or in one go:
di = einsum(images, dim_by, "batch height width channel, dim_value -> batch dim_value height width channel")
di.shape

torch.Size([64, 10, 128, 128, 3])

In [11]:
channels_last = torch.randn(64, 32, 32, 3) # (batch, height, width, channel)
B = torch.randn(32*32, 32*32)
h = w = 32
channels_flat = rearrange(channels_last, "b h w c -> b c (h w)")
channels_trans = einsum(channels_flat, B, "b c f, f g -> b c g")
channels_out = rearrange(channels_trans, "b c (h w) -> b h w c" , h=h, w=w)
channels_out.shape

torch.Size([64, 32, 32, 3])

In [1]:
import torch
import torch.nn.functional as F

x = torch.tensor([
    [[1.0, 2.0, 3.0],
     [4.0, 5.0, 6.0]],
    
    [[7.0, 8.0, 9.0],
     [10.0, 11.0, 12.0]]
])  # shape [2, 2, 3]

# 手写 softmax，沿 dim=2
x_max = x.max(dim=2, keepdim=True).values
x_stable = x - x_max
exp_x = torch.exp(x_stable)
softmax_x = exp_x / exp_x.sum(dim=2, keepdim=True)
softmax_x

tensor([[[0.0900, 0.2447, 0.6652],
         [0.0900, 0.2447, 0.6652]],

        [[0.0900, 0.2447, 0.6652],
         [0.0900, 0.2447, 0.6652]]])

In [7]:
1 - torch.triu(torch.ones(3,3),diagonal=1)

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [8]:
torch.tril(torch.ones(3, 3))

tensor([[1., 0., 0.],
        [1., 1., 0.],
        [1., 1., 1.]])

In [17]:
import numpy as np

np.random.randint(0,3,size=100)

array([1, 2, 1, 1, 2, 2, 0, 2, 2, 0, 0, 1, 0, 0, 2, 0, 1, 2, 1, 2, 2, 2,
       1, 0, 1, 0, 2, 0, 1, 2, 1, 2, 0, 0, 0, 0, 2, 1, 1, 1, 1, 1, 2, 0,
       1, 2, 0, 2, 1, 2, 2, 1, 2, 0, 1, 0, 1, 0, 0, 2, 0, 0, 2, 0, 0, 0,
       1, 1, 1, 0, 1, 1, 0, 2, 2, 0, 2, 0, 2, 0, 1, 1, 0, 1, 0, 0, 1, 1,
       1, 0, 2, 0, 2, 1, 2, 0, 1, 0, 2, 0])

In [19]:
327680000 / 64 / 256

20000.0

In [12]:
from collections.abc import Callable, Iterable
from typing import Optional
import torch
import math
import pandas as pd

class SGD(torch.optim.Optimizer):
    def __init__(self, params, lr=1e-3):
        if lr < 0:
            raise ValueError(f"Invalid learning rate: {lr}")
        defaults = {"lr": lr}
        super().__init__(params, defaults)
    def step(self, closure: Optional[Callable] = None):
        loss = None if closure is None else closure()
        for group in self.param_groups:
            lr = group["lr"] # Get the learning rate.
            for p in group["params"]:
                if p.grad is None:
                    continue
                state = self.state[p] # Get state associated with p.
                t = state.get("t", 0) # Get iteration number from the state, or initial value.
                grad = p.grad.data # Get the gradient of loss with respect to p.
                p.data -= lr / math.sqrt(t + 1) * grad # Update weight tensor in-place.
                state["t"] = t + 1 # Increment iteration number.
        return loss


lr_list = [1, 1e1, 1e2, 1e3]
loss_com = pd.DataFrame()
for lr in lr_list:
    loss_list = []
    weights = torch.nn.Parameter(5 * torch.randn((10, 10)))
    opt = SGD([weights], lr=lr)
    for t in range(10):
        opt.zero_grad() # Reset the gradients for all learnable parameters.
        loss = (weights**2).mean() # Compute a scalar loss value.
        loss_list.append(loss.cpu().item())
        loss.backward() # Run backward pass, which computes gradients.
        opt.step() # Run optimizer step.
    loss_com[f'{lr}'] = loss_list
loss_com
            

,1,10.0,100.0,1000.0
0,23.405771,23.690113,2.556360e+01,2.340510e+01
1,22.478905,15.161673,2.556360e+01,8.449238e+03
2,21.847601,11.176538,4.386019e+00,1.459316e+06
3,21.345963,8.744447,1.049673e-01,1.623334e+08
4,20.921177,7.083002,1.219966e-16,1.314900e+10
5,20.548603,5.872620,1.359728e-18,8.298526e+11
6,20.214415,4.952776,4.578682e-20,4.260193e+13
7,19.909958,4.232288,2.727552e-21,1.832917e+15
8,19.629385,3.654914,2.339870e-22,6.755740e+16
9,19.368532,3.183836,2.599856e-23,2.169343e+18


batch_size, vocab_size, context_length,

num_layers, d_model, num_heads

d_ff = 4 × d_model

参数: 

    Transformer block(

        RMSNorm: d_model

        Multi-head self-attention sublayer:(

            QKV projections: 3 * d_model^2

            output projection: d_model^2
        )
        Position-wise feed-forward:(

            W1 matrix multiply: d_model*d_ff = 4 * d_model^2

            W2 matrix multiply: 4 * d_model^2
        )
    )   Transformer block总计: num_layers * (d_model + 3 * d_model^2 + d_model^2 + 4 * d_model^2 + 4 * d_model^2) = num_layers * (d_model + 12 d_model^2)

    final RMSNorm: d_model

    output embedding: d_model * vocab_size

    参数总计: num_layers * (d_model + 12 d_model^2) + d_model + d_model * vocab_size

梯度和优化器状态:

    梯度：参数总计

    AdamW: m和v各自参数总计

    梯度和优化器状态总计: 3 * num_layers * (d_model + 12 d_model^2) + d_model + d_model * vocab_size

激活内存:

    对于每个batch：

        Transformer block(

            QKV: 3 * batch_size * context_length * d_model

            Q^T @ K matrix multiply: batch_size * num_heads * context_length^2

            softmax: batch_size * num_heads * context_length^2

            weighted sum of values: batch_size * num_heads * context_length * (d_model / num_heads) = batch_size * context_length * d_model

            output projection: batch_size * context_length * d_model

            Position-wise feed-forward:(

                W1 matrix multiply: batch_size * context_length * d_ff = 4 * batch_size * context_length * d_model

                SiLU: 4 * batch_size * context_length * d_model

                W2 matrix multiply: batch_size * context_length * d_model
            )
        )
        Transformer block总计激活内存: num_layers * (14 * batch_size * context_length * d_model + 2 * batch_size * num_heads * context_length^2)

        final RMSNorm: batch_size * context_length * d_model

        output embedding: batch_size * context_length * vocab_size
        
        cross-entropy on logits: batch_size * context_length * vocab_size
    
        单batch总计: num_layers * (14 * batch_size * context_length * d_model + 2 * batch_size * num_heads * context_length^2) + batch_size * context_length * d_model + 2 * batch_size * context_length * vocab_size
